## Section Headings Curation of El Salvador Policies

In this notebook there are a series of dictionaries and methods to curate section headings of El Salvador policies. Policies from El Salvador have a rather definite structure, so that the law text is organized under section headings. There are two kinds of sections, the ones that are general and that can be often found in many policies, and the ones which are specific. The sections headings which are more general often come with a whole range of name variants which makes the task of machine recognition difficult.

The goal of this notebook is to group all pretreatment methods that would harmonize sections heading to make the further processing machine friendly.

In [1]:
from pathlib import Path
import re, boto3, json, string, operator

### Dictionaries of particular vocabularies to help in the curation of section headings

In [4]:
# Most policies come with the final signatures. This is a piece of text that we want to be able to recognize. To make the
# detection of signatures easier, this dictionary contain the most common terms that can be found in these lines of text.
official_positions = {"ALCALDE" : 0,
"Alcalde" : 0,
"MINISTRA" : 0,
"Ministra" : 0,
"MINISTRO" : 0,
"Ministro" : 0,
"PRESIDENTA" : 0,
"Presidenta" : 0,
"PRESIDENTE" : 0,
"Presidente" : 0,
"REGIDOR" : 0,
"Regidor"  : 0,
"REGIDORA" : 0,
"regidora" : 0,
"SECRETARIA" : 0,
"Secretaria" : 0,
"SECRETARIO" : 0,
"Secretario" : 0,
"SINDICA" : 0,
"Sindica" : 0,
"SINDICO" : 0,
"Sindico" : 0,
"VICEPRESIDENTA" : 0,
"Vicepresidenta" : 0,
"VICEPRESIDENTE" : 0,
"Vicepresidente" : 0
}
# This dictionary contains some correspondences among different text headings. This is under development and needs further
# improvement.The idea is to merge in a single name all the headings that point to the same conceptual concept. For example,
# "Definiciones" is a heading that can come alone or together with other terms so it can appear as "Definiciones básicas" or
# "Definiciones generales". With the dictionary we can fetch all headings that contain the word "Definiciones" and change the
# heading to "Definiciones".
merges = {
    "DEFINICIONES" : "DEFINICIONES",
    "DISPOSICIONES FINALES" : "DISPOSICIONES GENERALES",
    "DISPOSICIONES GENERALES" : "DISPOSICIONES GENERALES",
    "DISPOSICIONES PRELIMINARES" : "DISPOSICIONES PRELIMINARES",
    "DISPOSICIONES REGULADORAS" : "DISPOSICIONES ESPECIALES",
    "DISPOSICIONES RELATIVAS" : "DISPOSICIONES ESPECIALES",
    "DISPOSICIONES REGULADORAS" : "DISPOSICIONES ESPECIALES",
    "DISPOSICIONES ESPECIALES" : "DISPOSICIONES ESPECIALES",
    "DISPOSICIONES TRANSITORIAS" : "DISPOSICIONES GENERALES"
}
# Eventhough the general gramar rule in Spanish is not to accent uppercase, there are many cases where a word in a heding might
# appear accented. This is a dictionary to armonize all headings without accents. The list is rather comprehensive, but there is
# still room for improvement.
# If we find some bug beyond simple misspelling which will be solved by spell checker, we can include it here. The example is in
# the first row with "ACTIVIDADESUSOS" which was found several times in headings.
accent_marks_bugs = {"ACTIVIDADESUSOS" : "ACTIVIDADES DE USOS",
    "ACUÍFERA" : "ACUIFERA",
    "ACUÍFERO" : "ACUIFERO",
    "ÁMBITO" : "AMBITO",
    "APLICACIÓN" : "APLICACION",
    "APROBACIÓN" : "APROBACION",
    "ARBÓREO" : "ARBOREO",
    "ARBÓREOS" : "ARBOREOS",
    "ARBORIZACIÓN" : "ARBORIZACION",
    "ÁREA" : "AREA",
    "ÁREAS" : "AREAS",
    "AUTORIZACIÓN" : "AUTORIZACION",
    "CAPTACIÓN" : "CAPTACION",
    "CLASIFICACIÓN" : "CLASIFICACION",
    "COMERCIALIZACIÓN" : "COMERCIALIZACION",
    "COMPENSACIÓN" : "COMPENSACION",
    "CONSERVACIÓN" : "CONSERVACION",
    "CONSTRUCCIÓN" : "CONSTRUCCION",
    "CONTAMINACIÓN" : "CONTAMINACION",
    "COORDINACIÓN" : "COORDINACION",
    "DECLARACIÓN" : "DECLARACION",
    "DEFINICIÓN" : "DEFINICION",
    "DEFORESTACIÓN" : "DEFORESTACION",
    "DELEGACIÓN" : "DELEGACION",
    "DESCRIPCIÓN" : "DESCRIPCION",
    "DESTINOS" : "DESTINO",
    "DISPOSICIÓN" : "DISPOSICION",
    "EDIFICACIÓN" : "EDIFICACION",
    "EDUCACIÓN" : "EDUCACION",
    "EJECUCIÓN" : "EJECUCION",
    "ELABORACIÓN" : "ELABORACION",
    "EVALUACIÓN" : "EVALUACION",
    "EXPLOTACIÓN" : "EXPLOTACION",
    "FISCALIZACIÓN" : "FISCALIZACION",
    "GALERÍA" : "GALERIA",
    "GESTIÓN" : "GESTION",
    "HÍDRICA" : "HIDRICA",
    "HÍDRICO" : "HIDRICO",
    "HÍDRICAS" : "HIDRICAS",
    "INFORMACIÓN" : "INFORMACION",
    "INFRACCIÓN" : "INFRACCION",
    "INTEGRACIÓN" : "INTEGRACION",
    "JURISDICCIÓN" : "JURISDICCION",
    "MULTAS" : "MULTA",
    "PARTICIPACIÓN" : "PARTICIPACION",
    "PLANIFICACIÓN" : "PLANIFICACION",
    "PLANTACIÓN" : "PLANTACION",
    "PREVENCIÓN" : "PREVENCION",
    "PROTECCIÓN" : "PROTECCION",
    "RECOLECCIÓN" : "RECOLECCION",
    "RECUPERACIÓN" : "RECUPERACION",
    "RÉGIMEN" : "REGIMEN",
    "REGULACIÓN" : "REGULACION",
    "RESOLUCIÓN" : "RESOLUCION",
    "RESTAURACIÓN" : "RESTAURACION",
    "SANCIÓN" : "SANCION",
    "SECCIÓN" : "SECCION",
    "SÍNDICA" : "SÍNDICA",
    "SÍNDICO" : "SINDICO",
    "SITUACIÓN" : "SITUACION",
    "SÓLIDO" : "SOLIDO",
    "SÓLIDOS" : "SOLIDOS",
    "TÉCNICA" : "TECNICA",
    "TÉCNICAS" : "TECNICAS",
    "TÉCNICO" : "TÉCNICO",
    "TÉCNICOS" : "TECNICOS",
    "TRÁMITE" : "TRAMITE",
    "TRÁMITES" : "TRAMITES",
    "TURÍSTICA" : "TURISTICA",
    "TURÍSTICO" : "TURISTICO",
    "ZONIFICACIÓN" : "ZONIFICACION"}

### Connection to the AWS S3 bucket
To effectively run this cell you need Omdena's credentials. Please keep them local and do not sync them in GitHub repos nor cloud drives. Before doing anything with this json file, please think of security!!

In [23]:
json_folder = Path("./credentials/")
filename = "Omdena_key.json"
file = json_folder / filename

with open(file, 'r') as f:
    key_dict = json.load(f) 

for key in key_dict:
    KEY = key
    SECRET = key_dict[key]

s3 = boto3.resource(
    service_name = 's3',
    region_name = 'us-east-2',
    aws_access_key_id = KEY,
    aws_secret_access_key = SECRET
)

### Functions and regular expressions

In [125]:

# Function to calculate the uppercase ratio in a string. It is used to detect section headings
def uppercase_ratio(string):
    return(len(re.findall(r'[A-Z]',string))/len(string))

# Regular expression to clear html tags (here is basically to remove the page tags)
cleanr = re.compile(r'<.*?>')
# Te function to clear html tags
def clean_html_tags(string):
  return cleanr.sub('', string)

# Function to remove the last lines of a document, the ones that contain the signatures of the officials. It depends on the
# dictionary "official_positions"
def remove_signatures(line):
    signature = False
    for key in official_positions:
        if key in line:
            signature = True
            break
    return signature

# Function to change accented words by non-accented counterparts. It depends on the dictionary "accent_marks_bugs" 
def remove_accents(line):
    words = line.split()
    for word in words:
        if word in accent_marks_bugs:
            line = line.replace(word, accent_marks_bugs[word])
    return line

# Function to merge headlines expressing the same concept in different words. It depends on the dictionary "merges"
def merge_concepts(line):
    for key in merges:
        if key in line:
            line = merges[key]
            break
    return line

# Function to add items to the dictionary with duplicate removal
def add_to_dict(string, dictionary, dupl_dict):
    if string in dupl_dict or string == None:
        pass
    else:
        dupl_dict[string] = 0
        if string in dictionary:
            dictionary[string] = dictionary[string] + 1
        else:
            dictionary[string] = 1
    return dictionary

# Regular expression to clear punctuation from a string
clean_punct = re.compile('[%s]' % re.escape(string.punctuation))
# Regular expression to clear words that introduce unnecessary variability to headings. Some regex still not work 100% we need
# to improve them.
clean_capitulo = re.compile(r'(APARTADO\s\S*)|(APARTADO\s)|(\bART\s\S*)|(\bART\s\d*)|(\bArt\s\S*)|(Capítulo\s\S*)|(CAPITULO\s\S*)|(CAPITULO\S*)|(CAPÍTULO\s\S*)|(CAPITULÓ\s\S*)|(CAPITULOS\s\S*)|(CAPITUO\s\S*)|(CATEGORIA\b)|(CATEGORÍA\b)|(SUBCATEGORIA\b)|(SUBCATEGORÍA\b)|(TITULO\s\S*)|(TÍTULO\s\S*)')
# Function to remove 1. unwanted words; 2. punctuation; 3. leading white spaces
def clean_headings(string):
    string = clean_capitulo.sub('', string)
    clean_string = clean_punct.sub('', string).rstrip().lstrip()
    if clean_string != "":
        return clean_string
    else:
        pass


### Pipeline to process files from S3 bucket
By executing this cell you will go through all policies in El Salvador and process section headings that will be saved in a dictionary. This should be merged with the notebook that builds up the final json files out of plain txt files.

In [126]:
folder = "text-extraction/"
filename = "00a55afe4f55256567397a68df5d7f97e642480b" # This is only if you want to test on a single file
bag_of_words = {}

i = 0
for obj in s3.Bucket('wri-latin-talent').objects.all().filter(Prefix='text-extraction'):
    if folder in obj.key and obj.key.replace(folder, "") != "": # and filename in obj.key # Un comment the previous string to run the code just in one sample document.
#         print(i, "**", obj.key)
        file = obj.get()['Body'].read().decode('utf-8')  #get the file from S3 and read the body content
        lines = file.split("\n") # Split by end of line and pipe lines into a list
        duplicates_dict = {} #Sometimes the same heading can be found more than once in a document. This will help on removing them
        for line in lines:
            if uppercase_ratio(clean_html_tags(line)) > 0.6 and len(line) > 6:
                if remove_signatures(line):
                    break
                else:
                    line = clean_html_tags(line)
                    line = remove_accents(line)
                    line = merge_concepts(line)
                    clean_line = clean_headings(line)
                    bag_of_words = add_to_dict(clean_line, bag_of_words, duplicates_dict)
#                 print("--", line)
#             s3.Object('wri-latin-talent', key).put(Body = content)#This will save all the contents in the string variable "content" into a txt file in the Pre-processed folder
        i += 1

#### Short summary

In [127]:
print("After preprocessing there are {} different headings in El Salvador policies".len(bag_of_words))
print("{} documents have been processed"i)

2460
349


316

#### Dictionary items sorted by occurrence

In [128]:
dict( sorted(bag_of_words.items(), key=operator.itemgetter(1),reverse=True))

{'CONSIDERANDO': 316,
 'POR TANTO': 268,
 'DISPOSICIONES GENERALES': 156,
 'DEFINICIONES': 103,
 'DECRETA': 100,
 'DISPOSICIONES PRELIMINARES': 51,
 'OBJETO': 42,
 'AMBITO DE APLICACION': 41,
 'OBLIGACIONES Y PROHIBICIONES': 41,
 'PROHIBICIONES': 40,
 'DE LAS INFRACCIONES Y SANCIONES': 30,
 'INFRACCIONES Y SANCIONES': 29,
 'OBLIGACIONES': 28,
 'DE LAS SANCIONES': 28,
 'VIGENCIA': 28,
 'DE LOS PERMISOS': 28,
 'AUTORIDAD COMPETENTE': 27,
 'MEDIDAS PREVENTIVAS': 25,
 'EL ÓRGANO EJECUTIVO EN EL RAMO DE MEDIO AMBIENTE Y RECURSOS NATURALES': 23,
 'INFRACCIONES SANCIONES Y RECURSOS': 20,
 'DE LAS PARCELACIONES': 18,
 'PROCEDIMIENTO': 18,
 'DE LOS USOS DEL SUELO': 17,
 'DEL PROCEDIMIENTO': 17,
 'COMPETENCIA': 16,
 'FORMULACIÓN APROBACION Y CONTENIDO DE LOS PLANES PARCIALES': 16,
 'DE LAS ZONAS DE PROTECCION': 16,
 'COORDINACION INTERINSTITUCIONAL': 14,
 'DE LOS TRAMITES Y SERVICIOS': 14,
 'DE LA RESPONSABILIDAD DE PROFESIONALES EMPRESAS PROPIETARIOS E INSTITUCIONES DE SERVICIO': 14,
 'DE LOS P

#### Dictionary items sorted by heading text

In [123]:
for k in sorted(bag_of_words):
    print(k, ":", bag_of_words[k])

007 IPOL 39 SENNA A SOUL On SOU TE ESA DO UORONAIIOS E ClEÓ SEAS IO0U SEXO : 1
090115 JUEQIN OJUBILUEYODSADAd : 1
1 DE LAS SANCIONES AUTORIDADES COMPETENTES PROCEDIMIENTOS Y RECURSOS 1 : 1
1 INFRACCIONES LEVES : 1
1 LIMITE CAN Y ESTATAL : 1
11503 CONSIDERADOS EN PTU : 1
2   FUNCIONES Y ATRIBUCIONES DE LA MUNICIPALIDAD : 1
2 ORDENANZA REGULADORA DE USO DE SUELOS DEL y e MUNICIPIO DE SANTA TECLA a : 1
46 PROHIBICIONES : 1
7 ADAPTACIÓN AL CAMBIO CLIMÁTICO 7 : 1
7 DE LA ADAPTACIÓN AL CAMBIO CLIMÁTICO 7 : 1
7 NIVELES O PISOS EN ADELANTE 40 TASAS : 3
A A o a ESPERAN E REINA AA PERA Ap ARA A EARL AE AO AOL lus cord 1016 ¡BA BOO IR URIA A BORIS CURS CANINO AA UA RUANO PATAEIUAS AA : 1
A ASA IAN : 1
A DEL OBJETO DE LA ORDENANZA : 1
A TE AAA : 1
AA PO ER : 1
ACCESO A LA PESCA Y LA ACUICULTURA : 1
ACCESO PROTECCION Y APROVECHAMIENTO DE LA DIVERSIDAD BIOLÓGICA : 1
ACCION EJECUTIVA : 1
ACCIONES CONJUNTAS E INSTITUCIONALES : 1
ACCIONES DE POLÍTICA PÚBLICA : 1
ACCIONES DE PREVENCION Y PROTECCIÓN : 1


DE LA RECOLECCION TRANSPORTE Y DISPOSICION FINAL DE LOS DESECHOS SOLIDOS : 3
DE LA RECOLECCION TRANSPORTE Y DISPOSICION FINAL DE LOS DESECHOS SOLIDOS Y MANEJO ADECUADO DE LAS AGUAS RESIDUALES : 1
DE LA RECOLECCION TRANSPORTE Y DISPOSICION FINAL DE LOS DESECHOS SÓLIDOS : 1
DE LA REDUCCION DESDE EL ORIGEN : 3
DE LA REDUCCIÓN DESDE EL ORIGEN : 3
DE LA REINCIDENCIA : 1
DE LA RESOLUCION : 5
DE LA RESOLUCIÓN : 1
DE LA RESPONSABILIDAD DE LOS AUTORES : 1
DE LA RESPONSABILIDAD DE LOS PROFESIONALES EMPRESAS PROPIETARIOS E INSTITUCIONES DE SERVICIO : 8
DE LA RESPONSABILIDAD DE PROFESIONALES EMPRESAS PROPIETARIOS E INSTITUCIONES DE SERVICIO : 14
DE LA SANCION DE MULTA : 1
DE LA SANCION DE MULTA Y CLASIFICACION DE LA INFRACCION : 9
DE LA SANCION DE MULTA Y CLASIFICACION DE LA INFRACCIÓN : 2
DE LA SANCION DE MULTA Y SU CLASIFICACION POR TIPO DE INFRACCION : 1
DE LA SANIDAD ANIMAL : 1
DE LA SANIDAD VEGETAL : 1
DE LA SEPARACION : 1
DE LA SEPARACIÓN : 2
DE LA SIEMBRA : 3
DE LA SIEMBRA DE ARBOLES : 1
DE

DISPOSICIONES GENERALES E : 1
DISPOSICIONES GENERALES OBJETO DE LA ORDENANZA : 1
DISPOSICIONES GENERALES PARA EL OSN EL OSA EL CIM Y EL OSARTEC LAS COMISIONES DIRECTIVAS Y LOS DIRECTORES TÉCNICOS : 1
DISPOSICIONES GENERALES TRANSITORIAS Y DEROGATORIAS : 1
DISPOSICIONES GENERALES Y VIGENCIA : 3
DISPOSICIONES PRELIMINARES : 47
DISPOSICIONES PRELIMINARES OBJETO : 4
DISPOSICIONES PRELIMINARES Y DEFINICIONES : 3
DISPOSICIONES PRELIMINARESY DEFINICIONES : 1
DISPOSICIONES PROCESALES : 9
DISPOSICIONES REGULADORAS DEL EJERCICIO DE LA CACERÍA DEPORTIVA EN EL SALVADOR : 1
DISPOSICIONES RELATIVAS A APLICACION AÉREA DE MADURANTES Y AGROQUÍMICOS : 1
DISPOSICIONES RELATIVAS A QUEMA EN CULTIVOS DE CAÑA DE AZÚCAR : 1
DISPOSICIONES RELATIVAS A RIEGO DE CULTIVOS DE CAÑA DE AZÚCAR : 1
DISPOSICIONES RELATIVAS AL SACRIFICIO DE ANIMALES PARA CONSUMO Y RESIDUOS RESULTANTES DE ESTAS ACTIVIDADES : 1
DISPOSICIONES SOBRE LA TALA DE ARBOLES : 1
DISPOSICIONES TRANSITORIAS : 8
DISPOSICIONES TRANSITORIAS Y FINALES : 

MECANISMOS DE SUBSANACIÓN : 1
MEDIDA PREVENTIVA : 3
MEDIDAS DE PREVENCION COMBATE Y CONTROL : 2
MEDIDAS DE PREVENCION Y CONTROL : 4
MEDIDAS DE PREVENCIÓN COMBATE Y CONTROL : 6
MEDIDAS PREVENTIVAS : 25
MEDIDAS PREVENTIVAS OBLIGACIONES Y PROHIBICIONES : 4
MEDIO AMBIENTE : 3
MEDIO AMBIENTE COSTEROMARINO AGUAS MARINAS Y SUS ECOSISTEMAS : 1
MEDIOS : 3
MERCADOS : 2
MIGUEL : 1
MILIROAIPDO ME IAUAIQUE : 1
MINAS BIENES INMUEBLES : 1
MODIFICACION DEROGATORIA Y VIGENCIA : 3
MODIFICACIÓN DEROGATORIA Y VIGENCIA : 2
MODO DE COBRAR LAS MULTA IMPUESTAS AL AMBITO FORESTAL : 3
MODO DE INICIAR EL PROCESO : 4
MODO DE SOLICITAR EL APOYO TÉCNICO : 3
MODO O FORMA DE INCREMENTAR LOS RECURSOS FORESTALES : 3
MOORE O TASA PO TT : 1
MORAZAN : 1
MUA TREZ DIE ¡COMP A TB ELIDAO DE 1 BO DE SUBLO : 1
MUNICIPAL : 1
MUNICIPAL DE LA CIUDAD DE ZACATECOLUCA DEPARTAMENTO DE LA PAZ : 1
MUNICIPIO DE JOCORO : 1
MUNICIPIO DE LAS VUELTAS : 1
MUNICIPIO DE NUEVA ESPARTA DEPARTAMENTO DE LA UNION : 1
MUNICIPIO DE NUEVA GUADALUPE DEP

RECOLECCION TRANSPORTE Y GESTION DE RESIDUOS SOLIDOS DE LA RECOLECCION : 1
RECOLECCION Y TRANSPORTE DE DESECHOS : 1
RECOLECION DE RESIDUOS SOLIDOS : 1
RECREACIÓN Y ENTRETENIMIENTO : 1
RECUPERACION AMBIENTAL : 1
RECUPERACION DEL MEDIO AMBIENTE DEL MUNICIPIO DE COMALAPA DEPARTAMENTO DE : 1
RECURSO AIRE : 1
RECURSO DE APELACION : 6
RECURSO DE APELACIÓN : 7
RECURSO DE REVISION : 1
RECURSO DE REVOCATORIA : 1
RECURSO FAUNA : 2
RECURSO FLORA : 1
RECURSO HIDRICO : 1
RECURSO HIDRICO EN EL MUNICIPIO DE MEANGUERA : 1
RECURSOS : 3
RECURSOS ECONOMICOS : 1
RECURSOS FINANCIEROS : 1
RECURSOS FORESTALES : 1
RECURSOS HIDRICOS : 2
RECURSOS HÍDRICOS : 2
RECURSOS NATURALES : 5
RECURSOS NATURALES DEL MUNICIPIO DE ARCATAO : 1
RECURSOS NATURALES DEL MUNICIPIO DE SAN SEBASTIÁN SALITRILLO : 1
RECURSOS NATURALES NO RENOVABLES : 2
RECURSOS NATURALES RENOVABLES : 1
REFORESTACION : 1
REFORESTACIÓN : 2
REFORMA DE LA ORDENANZA PARA LA PROTECCION DEL PATRIMONIO ARBOREO DEL MUNICIPIO DE NUEVO CUSCATLÁN : 1
REFORMAS : 8

#### Pipeline to process one file from HD folder
This is a pipeline to process a test file in a local folder.

In [91]:
data_folder = Path("../Documents_de_mostra/")
filename = "00a55afe4f55256567397a68df5d7f97e642480b.pdf.txt"
bag_of_words = {}

i = 0
file = data_folder / filename
with open(file, 'r', encoding = 'utf-8') as file:
    lines = file.readlines()
    duplicates_dict = {}
    for line in lines:
        line = clean_html_tags(line)
        if uppercase_ratio(line) > 0.6 and len(line) > 6:
            if remove_signatures(line):
                break
            else:
#                 print(line)
                line = remove_accents(line)
                clean_line = clean_headings(line)
                bag_of_words = add_to_dict(clean_line, bag_of_words, duplicates_dict)
#                 print(clean_line)
        i += 1
#     data = file.read().replace('\n', '')

DECRETO NÚMERO TRES

 CONSIDERANDO:

 ORDENANZA MUNICIPAL PARA LA IMPLEMENTACION DE LA POLITICA PÚBLICA “BOSQUES PARA SIEMPRE”, EN EL MUNICIPIO DE CANDELARIA DE LA FRONTERA. DEPARTAMENTO DE SANTA ANA.

 CAPITULO | DISPOSICIONES GENERALES.-

 OBJETO DE LA ORDENANZA.-

APLICACIÓN
APLICACION
 AMBITO DE APLICACION

 AUTORIDAD COMPETENTE

 FUNCIONES DE LA MUNICIPALIDAD

 FONDO FORESTAL.

 CONCEPTOS Y DEFINICIONES.

 CAPITULO ll PLAN CERO

 MANEJO SOSTENIBLE DE LOS BOSQUES, AREAS NATURALES Y ZONAS DE AMORTIGUAMIENTO DEL MUNICIPIO.-

 DE LOS BOSQUES.

 AREAS NATURALES

 ZONAS DE AMORTIGUAMIENTO

 VIGILANCIA DEL PATRIMONIO FORESTAL

 CAPITULO IV INCENTIVOS, DERECHOS Y DEBERES. PROMOCION DE INCENTIVOS

 DERECHOS DE LAS PERSONAS NATURALES Y JURIDICAS

 DEBERES DE LAS PERSONAS NATURALES Y JURIDICAS

 CAPITULO V OBLIGACIONES Y PROHIBICIONES

 OBLIGACIONES

 PROHIBICIONES.

 DE LAS SANCIONES.

 CAPITULO VI COMPETENCIA, PROCEDIMIENTOS Y RECURSOS.

 CAPITULO VII DISPOSICIONES GENERALES.



In [21]:
bag_of_words

{'DECRETO NÚMERO TRES': 1,
 'CONSIDERANDO': 1,
 'ORDENANZA MUNICIPAL PARA LA IMPLEMENTACION DE LA POLITICA PÚBLICA “BOSQUES PARA SIEMPRE” EN EL MUNICIPIO DE CANDELARIA DE LA FRONTERA DEPARTAMENTO DE SANTA ANA': 1,
 'DISPOSICIONES GENERALES': 1,
 'OBJETO DE LA ORDENANZA': 1,
 'AMBITO DE APLICACIÓN': 1,
 'AUTORIDAD COMPETENTE': 1,
 'FUNCIONES DE LA MUNICIPALIDAD': 1,
 'FONDO FORESTAL': 1,
 'CONCEPTOS Y DEFINICIONES': 1,
 'PLAN CERO': 1,
 'MANEJO SOSTENIBLE DE LOS BOSQUES AREAS NATURALES Y ZONAS DE AMORTIGUAMIENTO DEL MUNICIPIO': 1,
 'DE LOS BOSQUES': 1,
 'AREAS NATURALES': 1,
 'ZONAS DE AMORTIGUAMIENTO': 1,
 'VIGILANCIA DEL PATRIMONIO FORESTAL': 1,
 'INCENTIVOS DERECHOS Y DEBERES PROMOCION DE INCENTIVOS': 1,
 'DERECHOS DE LAS PERSONAS NATURALES Y JURIDICAS': 1,
 'DEBERES DE LAS PERSONAS NATURALES Y JURIDICAS': 1,
 'OBLIGACIONES Y PROHIBICIONES': 1,
 'OBLIGACIONES': 1,
 'PROHIBICIONES': 1,
 'DE LAS SANCIONES': 1,
 'COMPETENCIA PROCEDIMIENTOS Y RECURSOS': 1}